In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def triangular(x, parametros):
    a, b, c = parametros
    return max(min(((x-a)/(b-a)), ((c-x)/(c-b))), 0)

def gaussiana(x, parametros):
    media, dp = parametros
    return np.exp((-(x-media) **2)/(2*dp**2))

In [ ]:
def formatar_regras(regras, X, Y):
    for precedente, consequentes in regras.items():
        if len(X) == 1:
            precedente_str = f"SE {X[0]} = {precedente[0]}"
        else:
            precedente_parts = [f"{X[i]} = {precedente[i]}" for i in range(len(X))]
            precedente_str = "SE " + " E ".join(precedente_parts)

        consequente_parts = []
        consequentes_ordenados = sorted(consequentes.items(), key=lambda item: item[1], reverse=True)

        for nome, peso in consequentes_ordenados:
            peso_formatado = f"{nome}({peso:.2f})"
            consequente_parts.append(peso_formatado)

        consequente_str = f" ENTÃO {Y} = " + " + ".join(consequente_parts)
        print(precedente_str + consequente_str)

In [ ]:
class ConjuntoFuzzy(object):
    def __init__(self, nome, funcao, parametros, ponto_medio):
        self.nome = nome
        self.funcao = funcao
        self.parametros = parametros
        self.ponto_medio = ponto_medio

    def pertinencia(self, x):
        return self.funcao(x, self.parametros)

    def plot(self, eixo, intervalo):
        eixo.plot(intervalo, [self.pertinencia(k) for k in intervalo])

    def __str__(self):
        return "{}({})".format(self.nome, self.parametros)

def plot_conjuntos(eixo, conjuntos, intervalo):
    ticks = []
    ticks_names = []
    for nome, conj in conjuntos.items():
        conj.plot(eixo, intervalo)
        ticks.append(conj.ponto_medio)
        ticks_names.append(str(round(conj.ponto_medio, 2)) + "" + nome)

    eixo.set_xticks(ticks)
    eixo.set_xticklabels(ticks_names)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

x = [k for k in np.arange(-3 * np.pi, 3 * np.pi, 0.2)]
y = [k**2 * np.exp(np.sin(k)) for k in x]

df = pd.DataFrame({'x': x, 'y': y})
print(df)

plt.plot(x, y)
plt.xlabel("X")
plt.ylabel("Y")
plt.show()


In [ ]:
conjuntos = {
    'x': {
        'Esquerda': ConjuntoFuzzy("Esquerda", triangular, [-10, -5, 0], -5),
        'Centro': ConjuntoFuzzy("Centro", triangular, [-5, 0, 5 ], 0),
        'Direita': ConjuntoFuzzy("Direita", triangular, [0, 5, 10], 5),
    },
    'y': {
        'Alto': ConjuntoFuzzy("Alto", gaussiana, [175,2], 175),
        'Médio': ConjuntoFuzzy("Médio", gaussiana, [100, 2], 100),
        'Baixo': ConjuntoFuzzy("Baixo", gaussiana, [0, 2], 0),
    }
}

fix, eixo = plt.subplots(2)
plot_conjuntos(eixo[0], conjuntos['x'], [k for k in np.linspace(-10,10,100)])
plot_conjuntos(eixo[1], conjuntos['y'], [k for k in np.linspace(0, 175, 100)])

plt.tight_layout()


In [ ]:
from itertools import product

def inducao(dados, X, Y, conjuntos):
    dados_fuzzificados = []
    for indice in dados.index:
        linha = dados.loc[indice]
        linha_fuzzy = {}
        for variavel in conjuntos.keys():
            linha_fuzzy[variavel] = []
            for nome, conj in conjuntos[variavel].items():
                if conj.pertinencia(linha[variavel]) > 0:
                    linha_fuzzy[variavel].append(nome)
        dados_fuzzificados.append(linha_fuzzy)

    regras = {}
    for linha in dados_fuzzificados:
        x = [linha[k] for k in X]
        y = linha[Y]
        for precedente in product(*x):
            for consequente in y:
                if precedente not in regras:
                    regras[precedente] = {consequente: 1}
                else:
                    if consequente not in regras[precedente]:
                        regras[precedente][consequente] = 1
                    else:
                        regras[precedente][consequente] += 1

    for precedente in regras.keys():
        total = np.sum([regras[precedente][k] for k in regras[precedente].keys()])
        for k in regras[precedente].keys():
            regras[precedente][k] = regras[precedente][k] / total

    return regras


In [ ]:
def inferencia_regressao(dado, X, Y, conjuntos, regras):
    xfuzzy = {}
    for variavel in X:
        xfuzzy[variavel] = {}
        for nome, conj in conjuntos[variavel].items():
            pert = conj.pertinencia(dado[variavel])
            if pert > 0:
                xfuzzy[variavel][nome] = pert

    regras_ativadas = {}
    conj = []
    for variavel in X:
        conj.append([k for k in xfuzzy[variavel].keys()])

    for precedente in product(*conj):
        if precedente in regras:
            consequente = 0
            regras_ativadas[precedente] = {}
            for yfuzzy in regras[precedente].keys():
                consequente += conjuntos[Y][yfuzzy].ponto_medio * regras[precedente][yfuzzy]
            regras_ativadas[precedente]['centro'] = consequente

            ativacao = []
            for contador, conjunto in enumerate(precedente):
                ativacao.append(xfuzzy[X[contador]][conjunto])
            regras_ativadas[precedente]['ativacao'] = np.min(ativacao)

    denominador = 0
    numerador = 0
    for regra in regras_ativadas.keys():
        denominador += regras_ativadas[regra]['ativacao']
        numerador += regras_ativadas[regra]['ativacao'] * regras_ativadas[regra]['centro']

    return numerador / denominador


In [ ]:
regras = inducao(df, ['x'], 'y', conjuntos)
formatar_regras(regras, ['x'], 'y')

prever = lambda i: inferencia_regressao(df.iloc[i], ['x'], 'y', conjuntos, regras)
yy = [prever(k) for k in range(len(df.index))]

plt.plot(x,y)
plt.plot(x,yy)


In [ ]:
dp = 0.05
conjuntos = {
    'x': {
        'EsquerdaExt': ConjuntoFuzzy("EsquerdaExt", gaussiana, [-7.5, dp], -7.5),
        ...
    },
    'y': {
        'Alto': ConjuntoFuzzy("Alto", gaussiana, [175, 1], 175),
        ...
    }
}

fig, eixo = plt.subplots(2, figsize=[10,3])
plot_conjuntos(eixo[0], conjuntos['x'], [k for k in np.linspace(-10,10,100)])
plot_conjuntos(eixo[1], conjuntos['y'], [k for k in np.linspace(0,175,100)])
plt.tight_layout()

